# CLARITY: SemEval 2026 — Training Notebook

Works on **both Colab (GPU) and local**.

| Config | Model | VRAM | Expected Task2 F1 |
|--------|-------|------|-------------------|
| `deberta_v3_base.yaml` | DeBERTa-v3-base | ~6GB | ~0.48–0.56 |
| `deberta_v3_large.yaml` | DeBERTa-v3-large | ~16GB | ~0.56–0.65 |

## 1. Setup Environment

In [6]:
# Check GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    vram = getattr(props, 'total_memory', getattr(props, 'total_mem', 0))
    print(f"VRAM: {vram / 1e9:.1f} GB")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("MPS (Apple Silicon) available")
else:
    print("CPU only")

PyTorch: 2.10.0+cu128
CUDA: True
GPU: NVIDIA A100-SXM4-80GB
VRAM: 85.1 GB


In [7]:
import os, sys, subprocess

# Detect environment
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")

if IN_COLAB:
    REPO_URL = "https://github.com/wilsebbis/semeval.git"
    if not os.path.exists('/content/semeval'):
        !git clone {REPO_URL} /content/semeval
    else:
        !cd /content/semeval && git pull
    os.chdir('/content/semeval')
    # Install the package
    !pip install -e ".[dev]"
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir(os.path.join(os.getcwd(), '..'))

# Add src/ to path as fallback
src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"Working dir: {os.getcwd()}")

# Verify
from clarity.labels import EVASION_LABELS, CLARITY_LABELS, validate_labels
validate_labels()
print(f"✓ {len(EVASION_LABELS)} evasion labels, {len(CLARITY_LABELS)} clarity labels")

Environment: Colab
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 614 bytes | 614.00 KiB/s, done.
From https://github.com/wilsebbis/semeval
   1712731..5dc3b5d  main       -> origin/main
Updating 1712731..5dc3b5d
Fast-forward
 notebooks/train_colab.ipynb | 19 ++++++++++---------
 1 file changed, 10 insertions(+), 9 deletions(-)
Obtaining file:///content/semeval
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for clarity-semeval2026 (pyproject.toml) ... done
  Created wheel for clarity-semeval2026: filename=clarity_semeval2026-0.1.0-py3-none-any.whl size=3287 sha256=f6356b722fe966e7f943ee

## 2. Prepare Data

In [8]:
from pathlib import Path

if not Path('data/train.csv').exists():
    !git clone https://huggingface.co/datasets/ailsntua/QEvasion 2>/dev/null || echo "Already cloned"
    !bash scripts/prepare_data.sh
else:
    print("Data already prepared.")

Loaded train: 3448 rows, columns: ['title', 'date', 'president', 'url', 'question_order', 'interview_question', 'interview_answer', 'gpt3.5_summary', 'gpt3.5_prediction', 'question', 'annotator_id', 'annotator1', 'annotator2', 'annotator3', 'inaudible', 'multiple_questions', 'affirmative_questions', 'index', 'clarity_label', 'evasion_label']
Clarity distribution:
clarity_label
Ambivalent         2040
Clear Reply        1052
Clear Non-Reply     356
Evasion distribution:
evasion_label
Explicit               1052
Dodging                 706
Implicit                488
General                 386
Deflection              381
Declining to answer     145
Claims ignorance        119
Clarification            92
Partial/half-answer      79
Train split: 2930 rows
Dev split:   518 rows
Test: 308 rows
Done! Files saved to /content/semeval/data/
Data preparation complete.
  Train: /content/semeval/data/train.csv
  Dev:   /content/semeval/data/dev.csv
  Test:  /content/semeval/data/test.csv


In [9]:
import pandas as pd

train = pd.read_csv("data/train.csv")
dev = pd.read_csv("data/dev.csv")
print(f"Train: {len(train)} rows | Dev: {len(dev)} rows")
print(f"\nEvasion distribution:")
print(train["evasion_label"].value_counts().to_string())

Train: 2930 rows | Dev: 518 rows

Evasion distribution:
evasion_label
Explicit               894
Dodging                600
Implicit               415
General                328
Deflection             324
Declining to answer    123
Claims ignorance       101
Clarification           78
Partial/half-answer     67


## 3. Train

In [ ]:
# ── Choose config ────────────────────────────────────────
# CONFIG = "configs/deberta_v3_base.yaml"           # T4 OK
CONFIG = "configs/deberta_v3_large.yaml"        # A100/L4
# CONFIG = "configs/deberta_v3_base_no_weights.yaml"  # ablation
# ─────────────────────────────────────────────────────────

import yaml
with open(CONFIG) as f:
    cfg = yaml.safe_load(f)
print(f"Config: {CONFIG}")
for k, v in sorted(cfg.items()):
    print(f"  {k}: {v}")

Config: configs/deberta_v3_base.yaml
  alpha: 0.7
  batch_size: 8
  consistency_beta: 0.1
  dropout: 0.1
  epochs: 4
  fp16: False
  grad_accum: 2
  label_smoothing: 0.05
  lr: 2e-05
  max_length: 384
  model_name: microsoft/deberta-v3-base
  output_dir: checkpoints/deberta_v3_base
  patience: 3
  seed: 42
  task: evasion
  use_class_weights: True
  use_focal_loss: False
  warmup_ratio: 0.06
  weight_decay: 0.01


In [11]:
!python -m clarity.train \
    --config {CONFIG} \
    --data data/train.csv \
    --dev data/dev.csv \
    --task evasion

[2026-02-20 06:20:31] INFO clarity: Configuration: {'config': 'configs/deberta_v3_base.yaml', 'data': 'data/train.csv', 'dev': 'data/dev.csv', 'task': 'evasion', 'model_name': 'microsoft/deberta-v3-base', 'max_length': 384, 'batch_size': 8, 'grad_accum': 2, 'lr': 2e-05, 'weight_decay': 0.01, 'warmup_ratio': 0.06, 'epochs': 4, 'patience': 3, 'dropout': 0.1, 'seed': 42, 'output_dir': 'checkpoints/deberta_v3_base', 'use_focal_loss': False, 'focal_gamma': 2.0, 'use_class_weights': True, 'alpha': 0.7, 'consistency_beta': 0.1, 'num_workers': 0, 'fp16': False, 'device': None, 'label_smoothing': 0.05}
[2026-02-20 06:20:31] INFO clarity: Using device: cuda
config.json: 100% 579/579 [00:00<00:00, 2.69MB/s]
tokenizer_config.json: 100% 52.0/52.0 [00:00<00:00, 247kB/s]
spm.model: 100% 2.46M/2.46M [00:01<00:00, 1.84MB/s]
[2026-02-20 06:20:35] INFO clarity: Loaded tokenizer: microsoft/deberta-v3-base (fast=False)
[2026-02-20 06:20:35] INFO clarity: Loaded 2930 rows from data/train.csv
[2026-02-20 06:

## 4. Evaluate

In [12]:
import json
from pathlib import Path

output_dir = cfg["output_dir"]
metrics_path = Path(output_dir) / "metrics.json"

if metrics_path.exists():
    with open(metrics_path) as f:
        metrics = json.load(f)
    for m in metrics:
        ep = m.get('epoch', '?')
        ev = m.get('evasion_macro_f1', 0)
        cl = m.get('clarity_macro_f1', 0)
        print(f"  Epoch {ep}: Task2 F1={ev:.4f}, Task1 F1={cl:.4f}")
else:
    print("No metrics found — check training output above.")

AttributeError: 'str' object has no attribute 'get'

## 5. Predict

In [ ]:
CKPT = f"{output_dir}/best_model.pt"
DATA = "data/dev.csv"  # Change to data/test.csv for submission

!python -m clarity.predict \
    --ckpt {CKPT} \
    --data {DATA} \
    --out submissions/predictions.csv \
    --evaluate

In [ ]:
preds = pd.read_csv("submissions/predictions.csv")
print(f"Predictions: {len(preds)} rows")
print(preds["evasion_pred"].value_counts().to_string())

## 6. Ensemble (Optional)

Train 3 seeds, average logits → +1–3 F1 points.

In [ ]:
SEEDS = [42, 123, 2026]

for seed in SEEDS:
    out_dir = f"checkpoints/ensemble_seed{seed}"
    print(f"\n{'='*60}")
    print(f"Training seed {seed} -> {out_dir}")
    print(f"{'='*60}")
    !python -m clarity.train \
        --config {CONFIG} \
        --data data/train.csv \
        --dev data/dev.csv \
        --task evasion \
        --seed {seed} \
        --output_dir {out_dir}

In [ ]:
ckpts = [f"checkpoints/ensemble_seed{s}/best_model.pt" for s in SEEDS]
ckpts_str = " ".join(ckpts)

!python -m clarity.predict \
    --ckpt {ckpts[0]} \
    --ensemble_ckpts {ckpts_str} \
    --data data/dev.csv \
    --out submissions/ensemble_predictions.csv \
    --evaluate

## 7. Download (Colab only)

In [ ]:
if IN_COLAB:
    from google.colab import files
    files.download("submissions/predictions.csv")
else:
    print(f"Results: {os.path.abspath('submissions/predictions.csv')}")